# 📊 Exploratory Data Analysis (EDA)
Analyzing hotel booking data to understand customer behavior and cancellation trends.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"D:\Topic_13_Project\Topic_13_Retail_Store_Sales_Time_Series\data\raw\transactions.csv")

df.head(5)
df.info()
df.describe()
df.isnull().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB


date            0
store_nbr       0
transactions    0
dtype: int64

## Cleaning steps


In [2]:
def clean_transactions(raw: pd.DataFrame) -> pd.DataFrame:
    df = raw.copy()

    # 1. Standardize column names
    df.columns = [c.strip() for c in df.columns]

    required_cols = {"date", "store_nbr", "transactions"}
    missing = required_cols - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    # 2. Parse date
    df["date"] = pd.to_datetime(df["date"], errors="coerce")
    df = df.dropna(subset=["date"])

    # 3. Coerce numeric types
    df["store_nbr"] = pd.to_numeric(df["store_nbr"], errors="coerce")
    df["transactions"] = pd.to_numeric(df["transactions"], errors="coerce")

    # Drop rows with invalid store/transactions
    df = df.dropna(subset=["store_nbr", "transactions"])

    # store_nbr must be integer
    df["store_nbr"] = df["store_nbr"].astype(int)

    # 4. Handle invalid transactions (should be non-negative counts)
    df.loc[df["transactions"] < 0, "transactions"] = np.nan
    df = df.dropna(subset=["transactions"])
    df["transactions"] = df["transactions"].astype(int)

    # 5. Resolve duplicates: sum transactions per (date, store_nbr)
    df = (
        df.groupby(["date", "store_nbr"], as_index=False)["transactions"]
          .sum()
          .sort_values(["date", "store_nbr"])
    )

    # 6. Build complete panel (all dates x all stores)
    all_dates = pd.date_range(df["date"].min(), df["date"].max(), freq="D")
    all_stores = np.sort(df["store_nbr"].unique())

    panel = (
        pd.MultiIndex.from_product([all_dates, all_stores], names=["date", "store_nbr"])
          .to_frame(index=False)
          .merge(df, on=["date", "store_nbr"], how="left")
    )

    # 7. Fill missing with 0 and flag imputed
    panel["is_imputed"] = panel["transactions"].isna().astype(int)
    panel["transactions"] = panel["transactions"].fillna(0).astype(int)

    # Make date string for clean CSV (easy to submit/read)
    panel["date"] = panel["date"].dt.strftime("%Y-%m-%d")

    return panel


df_clean = clean_transactions(df)
df_clean.head()


,date,store_nbr,transactions,is_imputed
0,2013-01-01,1,0,1
1,2013-01-01,2,0,1
2,2013-01-01,3,0,1
3,2013-01-01,4,0,1
4,2013-01-01,5,0,1


In [3]:
# ====== QUICK CHECKS ======
print("Shape:", df_clean.shape)
print("Missing values:\n", df_clean.isnull().sum())
print("Duplicate (date, store_nbr):", df_clean.duplicated(subset=["date", "store_nbr"]).sum())
print("Unique stores:", df_clean["store_nbr"].nunique())
print("Unique days:", pd.to_datetime(df_clean["date"]).nunique())

df_clean.describe(include="all").T.head(10)


Shape: (91152, 4)
Missing values:
 date            0
store_nbr       0
transactions    0
is_imputed      0
dtype: int64
Duplicate (date, store_nbr): 0
Unique stores: 54
Unique days: 1688


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
date,91152,1688,2017-08-15,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN
store_nbr,91152.0,NaN,NaN,NaN,27.5,15.58587,1.0,14.0,27.5,41.0,54.0
transactions,91152.0,NaN,NaN,NaN,1552.121127,1034.916203,0.0,926.0,1329.0,1974.0,8359.0
is_imputed,91152.0,NaN,NaN,NaN,0.084079,0.277508,0.0,0.0,0.0,0.0,1.0


## Transaction Volume Distribution Analysis

Analyzing the statistical characteristics of transaction counts across stores and time.

In [4]:
# Transaction Volume Statistics (excluding imputed zeros)
from scipy import stats

df_actual = df_clean[df_clean['is_imputed'] == 0].copy()

# Basic statistics
mean_trans = df_actual['transactions'].mean()
median_trans = df_actual['transactions'].median()
std_trans = df_actual['transactions'].std()
min_trans = df_actual['transactions'].min()
max_trans = df_actual['transactions'].max()
q25_trans = df_actual['transactions'].quantile(0.25)
q75_trans = df_actual['transactions'].quantile(0.75)

# Distribution shape metrics
skewness_trans = stats.skew(df_actual['transactions'])
cv_trans = (std_trans / mean_trans) * 100

print("Transaction Volume Distribution (Actual Data Only):")
print("=" * 60)
print(f"Mean transactions/day: {mean_trans:,.0f}")
print(f"Median transactions/day: {median_trans:,.0f}")
print(f"Std deviation: {std_trans:,.0f}")
print(f"Min transactions: {min_trans:,}")
print(f"Max transactions: {max_trans:,}")
print(f"25th percentile: {q25_trans:,.0f}")
print(f"75th percentile: {q75_trans:,.0f}")
print(f"\nDistribution Characteristics:")
print(f"Coefficient of Variation: {cv_trans:.1f}%")
print(f"Skewness: {skewness_trans:.2f}")
print(f"Max/Mean ratio: {max_trans/mean_trans:.1f}x")

print(f"\n Interpretation:")
print(f"- High CV ({cv_trans:.1f}%) indicates SIGNIFICANT VARIABILITY across stores")
print(f"- Positive skewness ({skewness_trans:.2f}) shows right-tailed distribution")
print(f"- Busiest store has {max_trans/mean_trans:.1f}x more transactions than average")

Transaction Volume Distribution (Actual Data Only):
Mean transactions/day: 1,695
Median transactions/day: 1,393
Std deviation: 963
Min transactions: 5
Max transactions: 8,359
25th percentile: 1,046
75th percentile: 2,079

Distribution Characteristics:
Coefficient of Variation: 56.8%
Skewness: 1.52
Max/Mean ratio: 4.9x

 Interpretation:
- High CV (56.8%) indicates SIGNIFICANT VARIABILITY across stores
- Positive skewness (1.52) shows right-tailed distribution
- Busiest store has 4.9x more transactions than average


## Store-Level Transaction Heterogeneity

Analyzing how transaction volumes vary across different stores to understand traffic patterns.

In [5]:
# Store-level transaction analysis
store_trans = df_actual.groupby('store_nbr')['transactions'].agg(['mean', 'median', 'std', 'min', 'max', 'count'])
store_trans = store_trans.sort_values('mean', ascending=False)

# Top and bottom performers
top_5_stores = store_trans.head(5)
bottom_5_stores = store_trans.tail(5)

print("Top 5 Stores by Average Daily Transactions:")
print("=" * 60)
for idx, row in top_5_stores.iterrows():
    print(f"Store #{idx}: Avg={row['mean']:,.0f}, Median={row['median']:,.0f}, Max={row['max']:,.0f}")

print(f"\nBottom 5 Stores by Average Daily Transactions:")
print("=" * 60)
for idx, row in bottom_5_stores.iterrows():
    print(f"Store #{idx}: Avg={row['mean']:,.0f}, Median={row['median']:,.0f}, Max={row['max']:,.0f}")

# Heterogeneity metrics
busiest_store = store_trans.index[0]
quietest_store = store_trans.index[-1]
heterogeneity_ratio = store_trans['mean'].iloc[0] / store_trans['mean'].iloc[-1]

print(f"\n Store Heterogeneity Insights:")
print(f"- Busiest store (#{busiest_store}): {store_trans['mean'].iloc[0]:,.0f} avg transactions/day")
print(f"- Quietest store (#{quietest_store}): {store_trans['mean'].iloc[-1]:,.0f} avg transactions/day")
print(f"- Heterogeneity ratio: {heterogeneity_ratio:.1f}x difference between busiest and quietest")
print(f"- This indicates EXTREME STORE-LEVEL VARIATION in foot traffic")

Top 5 Stores by Average Daily Transactions:
Store #44: Avg=4,337, Median=4,170, Max=8,359
Store #47: Avg=3,897, Median=3,685, Max=7,727
Store #45: Avg=3,698, Median=3,515, Max=7,305
Store #46: Avg=3,572, Median=3,230, Max=8,001
Store #3: Avg=3,202, Median=3,100, Max=6,085

Bottom 5 Stores by Average Daily Transactions:
Store #22: Avg=751, Median=730, Max=2,412
Store #30: Avg=708, Median=701, Max=1,443
Store #35: Avg=671, Median=656, Max=1,676
Store #32: Avg=635, Median=615, Max=1,497
Store #26: Avg=635, Median=591, Max=2,184

 Store Heterogeneity Insights:
- Busiest store (#44): 4,337 avg transactions/day
- Quietest store (#26): 635 avg transactions/day
- Heterogeneity ratio: 6.8x difference between busiest and quietest
- This indicates EXTREME STORE-LEVEL VARIATION in foot traffic


## Weekly Seasonality Analysis

Analyzing day-of-week patterns to identify weekend and weekday traffic differences.

In [6]:
# Weekly pattern analysis
df_actual['date_dt'] = pd.to_datetime(df_actual['date'])
df_actual['day_of_week'] = df_actual['date_dt'].dt.day_name()
df_actual['is_weekend'] = df_actual['date_dt'].dt.dayofweek.isin([5, 6]).astype(int)

# Average transactions by day of week
dow_trans = df_actual.groupby('day_of_week')['transactions'].mean().reindex([
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'
])

# Calculate weekend uplift
weekday_avg = df_actual[df_actual['is_weekend'] == 0]['transactions'].mean()
weekend_avg = df_actual[df_actual['is_weekend'] == 1]['transactions'].mean()
weekend_uplift = ((weekend_avg - weekday_avg) / weekday_avg) * 100

print("Average Transactions by Day of Week:")
print("=" * 60)
for day, avg_trans in dow_trans.items():
    vs_weekday = ((avg_trans - weekday_avg) / weekday_avg) * 100
    print(f"{day:12s}: {avg_trans:,.0f} ({vs_weekday:+.1f}% vs weekday avg)")

print(f"\nWeekday vs Weekend Comparison:")
print("=" * 60)
print(f"Weekday average: {weekday_avg:,.0f} transactions")
print(f"Weekend average: {weekend_avg:,.0f} transactions")
print(f"Weekend uplift: {weekend_uplift:+.1f}%")

# Find peak day
peak_day = dow_trans.idxmax()
peak_value = dow_trans.max()

print(f"\n Weekly Seasonality Insights:")
print(f"- Peak day: {peak_day} with {peak_value:,.0f} avg transactions")
print(f"- Weekend shows {weekend_uplift:+.1f}% traffic boost vs weekdays")
print(f"- Strong WEEKLY SEASONALITY pattern detected")

Average Transactions by Day of Week:
Monday      : 1,636 (+1.4% vs weekday avg)
Tuesday     : 1,601 (-0.7% vs weekday avg)
Wednesday   : 1,621 (+0.6% vs weekday avg)
Thursday    : 1,550 (-3.9% vs weekday avg)
Friday      : 1,654 (+2.6% vs weekday avg)
Saturday    : 1,953 (+21.1% vs weekday avg)
Sunday      : 1,847 (+14.6% vs weekday avg)

Weekday vs Weekend Comparison:
Weekday average: 1,612 transactions
Weekend average: 1,900 transactions
Weekend uplift: +17.9%

 Weekly Seasonality Insights:
- Peak day: Saturday with 1,953 avg transactions
- Weekend shows +17.9% traffic boost vs weekdays
- Strong WEEKLY SEASONALITY pattern detected


In [7]:
# ====== EXPORT CLEANED DATA ======
output_path = "transactions_cleaned.csv"
df_clean.to_csv(output_path, index=False)
output_path


'transactions_cleaned.csv'